### Deep Learning Miniproject - Audio

AVS 8th Semester - Group 841

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import IPython.display as ipd

import torch
import torchaudio
from torch import nn
import torch.optim 

from datasets import load_dataset, DatasetDict, load_metric
from transformers import ASTFeatureExtractor, ASTForAudioClassification, ASTConfig, TrainingArguments, Trainer

In [ ]:
import wandb

### Parameters

In [ ]:
# data loading
AUDIO_DIR = "./data/"
CSV_DIR = "./data/metadata_compiled.csv"
FILE_TYPE = ".mp3"

# model
SAMPLING_RATE = 16000
BATCH_SIZE = 4
LEARNING_RATE = 5e-5
CHECKPOINT = 'MIT/ast-finetuned-audioset-10-10-0.4593'

In [ ]:
#.csv file loading
df = pd.read_csv(CSV_DIR)

#### 1. Explore the dataset through code

a. How many samples does the dataset contain?

In [ ]:
#Check no. samples
print(f'Number of samples : {df.shape[0]}')

b. How many classes? How many samples per class? Show a histogram of the number of intances per class

In [ ]:
print(f'Number of classes: {len(df["status"].unique())}.\n\
    Classes: {df["status"].unique()}\n\
    {pd.value_counts(df["status"], dropna=False)}')

In [ ]:
pd.value_counts(df['status'], dropna=False).plot.bar()
plt.show()

c. Play a random sample from each class

In [ ]:
# playing healthy
healthy = df[df['status'] == 'healthy'].sample()['uuid'].item()
path = AUDIO_DIR + healthy + FILE_TYPE
print(path)
y, sr = torchaudio.load(path)
ipd.Audio(y, rate=sr)

In [ ]:
# playing COVID-19
covid = df[df['status'] == 'COVID-19'].sample()['uuid'].item()
path = AUDIO_DIR + covid + FILE_TYPE
y, sr = torchaudio.load(path)
ipd.Audio(y, rate=sr)

In [ ]:
# playing symptomatic
symptomatic = df[df['status'] == 'symptomatic'].sample()['uuid'].item()
path =  AUDIO_DIR + symptomatic + FILE_TYPE
y, sr = torchaudio.load(path)
ipd.Audio(y, rate=sr)

d. Describe if/how you think the data distribution will affect training of a classifier

e. Decide what part of the dataset to use; all, some classes, some samples. Motivate your choice

#### 2. Use a neural network of your own chose to classify the dataset. Explain your choice and at least one alternative. Document your experiences:.

#### Audio Spectogram Transformer Implementation

##### Creating a Dataset class

We create a custom Dataset class to load our cough files. 

In Pytorch the Dataset class has to override the functions: **\__len__** and **\__getitem__**, where **\__len__** returns the amount of files in the dataset, and **\__getitem__** returns the file and label for each file index.


In [ ]:
class AudioDatatset(Dataset):
    def __init__(self, audio_dir, class_csv):
        self.audio_dir = audio_dir
        self.df = pd.read_csv(class_csv)

        self.audio_dir_list = os.listdir(self.audio_dir)

    def __len__(self):
        return len(self.audio_dir_list)
    
    def __getitem__(self, idx):
        #Loading the audio file
        audio_file_path = os.path.join(self.audio_dir, self.audio_dir_list[idx])
        waveform, sample_rate = ta.load(audio_file_path, normalize=True)
        
        #Transforming to mel spectogram
        transform = ta.transforms.MelSpectrogram(sample_rate, n_mels=32)
        mel_specgram = transform(waveform) 

        #Loading the label
        audio_file_name = self.audio_dir_list[idx].replace(FILE_TYPE, '')

        i =  self.df[ self.df['uuid']==audio_file_name].index.values
        label =  self.df["status"].loc[ self.df.index[i].values[0]]
        #Convert the label from string to a number. Healthy = 0, Symptomaic = 1, Covid = 2
        if label == 'healthy':
            label = 0
        elif label == 'symptomatic':
            label == 1
        else:
            label == 2


        return mel_specgram, label 


##### Testing the Dataset class

We can quickly create a an instance of the AudioDataset class and print out values for an item in the dataset.

In [ ]:
test = AudioDatatset(AUDIO_DIR, CSV_DIR)
print(test.__getitem__(0))
print(test.__len__())

##### Creating a DataModule class

Pytorch also has a DataModule class that loads the data from the Dataset class we just made. In this class we split the dataset into training, validation and testing with a **70/20/10** split. This DataModule class also allows us to set the batch size, number of workers and more for the training. Since we are using Pytorch lightning, we need to have the following functions: **prepare_data, setup, train_dataloader, val_dataloader and test_dataloader.**

In [ ]:
class DataModuleClass(pl.LightningDataModule):
    def __init__(self, batch_size):
        super().__init__()
        #self.transform = transforms.MelSpectrogram(sample_rate)
        self.batch_size = batch_size
        self.audio_files =  []

    def prepare_data(self):
        
        pass
        #Define steps that should be done
        #only on one GPU, like getting data

    def setup(self, stage=None):
        #Apply melSpectogram transform
        self.audio_files = AudioDatatset(AUDIO_DIR, CSV_DIR)

        #Splitting manually 
        audio_len = self.audio_files.__len__()
        train_size = round(audio_len * 0.7)
        val_size = round(audio_len * 0.2)
        test_size = audio_len - train_size - val_size
        
        self.train_data, self.val_data, self.test_data = random_split(self.audio_files, [train_size, val_size, test_size])


    def train_dataloader(self):
        return DataLoader(self.train_data, self.batch_size, num_workers=2, pin_memory=True, persistent_workers=True)       

    def val_dataloader(self):
        return DataLoader(self.val_data, self.batch_size, num_workers=2, pin_memory=True, persistent_workers=True)
    
    def test_dataloader(self):
        return DataLoader(self.test_data, self.batch_size, num_workers=2, pin_memory=True, persistent_workers=True)  


Vini trying model stuff

In [ ]:
# https://huggingface.co/docs/transformers/main/en/model_doc/audio-spectrogram-transformer#transformers.ASTConfig
from transformers import ASTFeatureExtractor, ASTForAudioClassification, ASTConfig

# must be the same -> model and tokenizer/feature extractor
# right now with default values

config = ASTConfig()

# basically tokenizer
# input normalization: mean = 0, std = 0.5

#feature_extractor = ASTFeatureExtractor(config, sampling_rate=sr, num_mel_bins=32, mean=0, std=0.5)

model = ASTForAudioClassification(config)

train_loader = DataModuleClass(BATCH_SIZE)

trainer = pl.Trainer(max_epochs=1, accelerator='gpu', devices=1, log_every_n_steps=25)

trainer.fit(model, train_loader)



##### Load soundfiles into dataset

In [ ]:
# must have metadata.csv with 'file_name' column to have also the features
# unsplitted dataset
dataset = load_dataset("audiofolder", data_dir=AUDIO_DIR, split="train")

In [ ]:
# splitting up the data : {training - 70%, validation - 20%, test - 10%}
# shuffle=True
train_testvalid = dataset.train_test_split(test_size=0.3)
test_valid = train_testvalid['test'].train_test_split(test_size=1/3)

dataset = DatasetDict({
    'train' : train_testvalid['train'],
    'test' : test_valid['test'],
    'valid' : test_valid['train']
})

In [ ]:
# getting the labels for classification
labels = list(df["status"].unique()[1:])

label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

num_classes = len(labels)

In [ ]:
# hyperparams again 
# TODO : move up
max_duration = 1

hidden_dim = 768

max_seq_length = max_duration * SAMPLING_RATE
max_frames = 49
max_epochs = 2

is_cuda = torch.cuda.is_available()

##### Training Model

In [ ]:
# use this for [RANDOM WEIGHTS] - no pretraining
config = ASTConfig(
    hidden_size=768, # default : 768
    num_hidden_layers=12, # default : 12
    hidden_dropout_prob= 0.0 # def.: 0.0
    attention_probs_dropout_prob=0.0 # def.: 0.0
)

# basically tokenizer
# input normalization: mean = 0, std = 0.5
feature_extractor = ASTFeatureExtractor(config, sampling_rate=SAMPLING_RATE, num_mel_bins=32, mean=0, std=0.5)

model = ASTForAudioClassification(config)
# weights must be the same for the model and the tokenizer/feature extractor 

##### Pretrained session

In [ ]:
# feature extractor and model
feature_extractor = ASTFeatureExtractor(
    CHECKPOINT
)

model = ASTForAudioClassification.from_pretrained(
    CHECKPOINT, 
    num_labels=len(labels),
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True
    )

In [ ]:
# map function
def preprocess(examples):
    audio_arr = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arr,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=max_seq_length,
        truncation=True,
        padding=True
    )
    
    label = [int(label2id[x]) for x in examples["label"]]
    
    inputs["label"] = label
    return inputs

### Tokenizing dataset

In [ ]:
# mapping datasets -> feature extraction
train_ = dataset['train'].map(preprocess, remove_columns=["audio"], batched=True)
valid_ = dataset['valid'].map(preprocess, remove_columns=["audio"], batched=True)
test_ = dataset['test'].map(preprocess, remove_columns=["audio"], batched=True)

### Training the model

In [ ]:
# setting the optimizer and the scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer)

In [ ]:
def compute_metrics(eval_pred):
    metrics = dict()

    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    accuracy_metric = load_metric('accuracy')
    precision_metric = load_metric('precision')
    recall_metric = load_metric('recall')
    f1_metric = load_metric('f1')

    metrics.update(accuracy_metric.compute(predictions=preds, references=labels))
    metrics.update(precision_metric.compute(predictions=preds, references=labels, average='weighted'))
    metrics.update(recall_metric.compute(predictions=preds, references=labels, average='weighted'))
    metrics.update(f1_metric.compute(predictions=preds, references=labels, average='weighted'))

    return metrics

### Setting up wandb

In [ ]:
# set the wandb project where this run will be logged
os.environ["WANDB_PROJECT"]="cough-project"

# save your trained model checkpoint to wandb
os.environ["WANDB_LOG_MODEL"]="true"

# turn off watch to log faster
os.environ["WANDB_WATCH"]="false"

In [ ]:
# the hyperparams for Trainer
training_arg = TrainingArguments(
    output_dir="output",
    report_to="wandb",
    learning_rate=LEARNING_RATE,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    #num_train_epochs= max_epochs,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    max_steps=1000,
    logging_steps=50,
    eval_steps=200, 
    eval_accumulation_steps=1, 
    load_best_model_at_end=True,
    warmup_steps=50,
    save_total_limit=2,
    #metric_for_best_model='accuracy'
    )

# defining trainer
trainer = Trainer(
    model=model,
    args=training_arg,
    train_dataset=train_,
    eval_dataset=valid_,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
    optimizers = (optimizer, scheduler)
)

In [ ]:
# training session
trainer.train()

In [ ]:
wandb.finish()

In [ ]:
# evaluation
predictions = trainer.predict()

In [ ]:
# saving model weigths into files
model.save_pretrained('./saved_model/')

In [ ]:
trainer.evaluate()

a. Discuss at least four relevant hyper-parameters

Learning rate : \
Epoch Numer : \
Batch Size : \
Optimizer: \
Layer Number : [??]

In [ ]:
# learning rate
# epoch number
# mini-batch size

# I don't know what is the 4th one

b. Experiment with the effect of different batch sizes

c. Experiment with the effect of different learning rates

d. Experiment with different number of network layers

e. Implement at least two data agumentation techniques

f. Discuss what influences the memory use of a solution such as yours. What can be done to reduce this?